In [1]:
import utils as ui ##needed to import my iris
import iris
import numpy as np
import time
import os
import sys

In [2]:
def fill_array(tc,xc,yc,zc):
    valmax = 30
    valmin = 0
    zmin = min(zc.points)
    zmax = max(zc.points)

    t = (tc.points/(365.*24.))*(2.*np.pi)
    x = (xc.points/360.) *(2.*np.pi)
    y = (yc.points/360.) *(2.*np.pi)
    z = (zc.points - zmax)/(zmax-zmin) 
    
    #temp = (valmax-valmin) * np.einsum('i,j,k,l',1+0.*np.sin(t), np.sin(x), np.cos(y), 1+0.*z)
    temp = (valmax-valmin) * np.einsum('i,j,k,l', np.sin(t), np.sin(x), np.cos(y), z)

    return temp

In [3]:
#!/usr/bin/env python
import iris
import numpy as np
iris.FUTURE.netcdf_no_unlimited = True

#t coordinate in hours: 8760 ~ 1yr
def generate_cube(tmin = 1, tmax = 24*7,tsamp = 24*7,
                xmin = -179, xmax = 180, xsamp = 360,
                ymin = -89, ymax = 89, ysamp = 178,
                zmin = 0, zmax = 7000, zsamp = 2):

    #Create coordinates:
    tco = iris.coords.DimCoord(np.linspace(tmin,tmax,tsamp), standard_name='time', units='hours since 2000-01-01 00:00')
    tco.guess_bounds()

    xco = iris.coords.DimCoord(np.linspace(xmin,xmax,xsamp), standard_name='longitude', units='degree')
    xco.guess_bounds()

    yco = iris.coords.DimCoord(np.linspace(ymin,ymax,ysamp), standard_name='latitude', units='degree')
    yco.guess_bounds()

    zco = iris.coords.DimCoord(np.linspace(zmin,zmax,zsamp), standard_name='height', units='metres')
    zco.guess_bounds()

    #Create cube:
    c1 = iris.cube.Cube(fill_array(tco,xco,yco,zco), standard_name='air_temperature', units='celsius')
    c1.add_dim_coord(tco, 0)
    c1.add_dim_coord(xco, 1)
    c1.add_dim_coord(yco, 2)
    c1.add_dim_coord(zco, 3)

    cube = c1
    return cube

In [ ]:
def my_plot(some_cube):

#PLOT
#Adapted from http://scitools.org.uk/iris/docs/latest/examples/General/anomaly_log_colouring.html

    # Construct a plot title
    plot_title = some_cube.name()
    


    # Create an Axes, specifying the map projection.
    plt.axes(projection=ccrs.Mollweide())
    #plt.axes(projection=ccrs.Stereographic())

    # Make a pseudocolour plot using this colour scheme.
    # Use a standard colour map which varies blue-white-red.
    # For suitable options, see the 'Diverging colormaps' section in:
    # http://matplotlib.org/examples/color/colormaps_reference.html
    anom_cmap = 'bwr'
    mesh = iplt.pcolormesh(some_cube, cmap=anom_cmap)

    # Add a colourbar, with extensions to show handling of out-of-range values.
    bar = plt.colorbar(mesh, orientation='horizontal', extend='both')

    # Set some suitable fixed "logarithmic" colourbar tick positions.
    #tick_levels = [-3, -2, -1, 0.0, 1, 2, 3]
    #bar.set_ticks(tick_levels)

    # Modify the tick labels so that the centre one shows "+/-<minumum-level>".        
    #minimum_level = 0.0
    #maximum_level = 30
    #tick_levels[3] = r'$\pm${:g}'.format(minimum_level)
    #bar.set_ticklabels(tick_levels)

    # Label the colourbar to show the units.
    bar.set_label('[{}]'.format(time_mean.units))

    # Add coastlines and a title.
    plt.gca().coastlines()
    plt.title(plot_title)

    # Display the result.
    iplt.show()


In [ ]:
#TEST: CALCULATE THE MEAN
import cartopy.crs as ccrs
import iris.coord_categorisation
import matplotlib.pyplot as plt
import iris.plot as iplt
import iris.quickplot as qplt

# Enable a future option, to ensure that the netcdf load works the same way
# as in future Iris versions.
iris.FUTURE.netcdf_promote = True

# Load a sample air temperatures sequence.
temperatures = generate_cube(tmin = 24, tmax = 24*7,tsamp = 24*7,
                xmin = -170, xmax = 170, xsamp = 36,
                ymin = -80, ymax = 80, ysamp = 18,
                zmin = 0, zmax = 7000, zsamp = 2)


# Calculate the mean:
time_mean = temperatures.collapsed(['time','height'], iris.analysis.MEAN)

# Plot:
my_plot(time_mean)

In [ ]:
#TEST CONCATENATION AND THEN MEAN

#Each cube will span from t_a[i] to t_b[i] with t_num_samp in between
#Total span of the cubes: t_a[0] to t_b[-1]

t_a = range(1 , 24*800-23,24)
t_b = range(24, 24*800,24)
t_num_samp = 24

cubes = []
for t_init,t_end in zip(t_a,t_b):
    #print(t_init,t_end)
    cubes.append(generate_cube(tmin = t_init, tmax = t_end,tsamp = t_num_samp,
                xmin = -170, xmax = 170, xsamp = 36,
                ymin = -80, ymax = 80, ysamp = 18,
                zmin = 0, zmax = 7000, zsamp = 2))

loaded_cubes = iris.cube.CubeList(cubes)

disordered = iris.cube.CubeList([loaded_cubes[6],loaded_cubes[0], loaded_cubes[5],loaded_cubes[1],
              loaded_cubes[4],loaded_cubes[2], loaded_cubes[3]])

concatenated = disordered.concatenate()
print(concatenated)

concatenated = loaded_cubes.concatenate()
print(concatenated)

concat_mean = concatenated[0].collapsed(['time','height'], iris.analysis.MEAN)

# Plot:
my_plot(concat_mean)

In [ ]:
#TEST MEAN AND THEN CONCATENATION

#Each cube will span from t_a[i] to t_b[i] with t_num_samp in between
#Total span of the cubes: t_a[0] to t_b[-1]

t_a = range(1 , 24*800-23,24)
t_b = range(24, 24*800,24)
t_num_samp = 24

cubes = []
for t_init,t_end in zip(t_a,t_b):
    #print(t_init,t_end)
    cubes.append(generate_cube(tmin = t_init, tmax = t_end,tsamp = t_num_samp,
                xmin = -170, xmax = 170, xsamp = 36,
                ymin = -80, ymax = 80, ysamp = 18,
                zmin = 0, zmax = 7000, zsamp = 2).collapsed(['time','height'], iris.analysis.MEAN))

loaded_cubes = iris.cube.CubeList(cubes)
merged_cubes = loaded_cubes.merge()
print(merged_cubes)
mean_merge_mean = merged_cubes[0].collapsed(['time','height'], iris.analysis.MEAN)
my_plot(mean_merge_mean)

In [ ]:
print(mean_merge_mean)

In [ ]:
print(concat_mean)

In [ ]:
iris.save(mean_merge_mean,'./shitload_of_data/mean_merge_mean.nc')
iris.save(concat_mean,'./shitload_of_data/concat_mean.nc')

In [ ]:
#Save cubes after applying the mean - This simulates what we have after the map phase

#Each cube will span from t_a[i] to t_b[i] with t_num_samp in between
#Total span of the cubes: t_a[0] to t_b[-1]

t_a = range(1 , 24*701-23,24)
t_b = range(24, 24*701,24)
t_num_samp = 24

cubes = []
i=0
for t_init,t_end in zip(t_a,t_b):
    #print(t_init,t_end)
    i += 1
    cube = generate_cube(tmin = t_init, tmax = t_end,tsamp = t_num_samp,
                xmin = -170, xmax = 170, xsamp = 36,
                ymin = -80, ymax = 80, ysamp = 18,
                zmin = 0, zmax = 7000, zsamp = 2).collapsed(['time','height'], iris.analysis.MEAN)
    iris.save(cube,'./shitload_of_data/mean_cube_'+str(i)+'.nc')


In [ ]:
#Make sure that all these loads are merged in one single cube:

cubes1 = iris.load('./shitload_of_data/mean_cube_1*.nc')
cubes2 = iris.load('./shitload_of_data/mean_cube_2*.nc')
cubes3 = iris.load('./shitload_of_data/mean_cube_3*.nc')
cubes4 = iris.load('./shitload_of_data/mean_cube_4*.nc')
cubes5 = iris.load('./shitload_of_data/mean_cube_5*.nc')
cubes6 = iris.load('./shitload_of_data/mean_cube_6*.nc')
cubes7 = iris.load('./shitload_of_data/mean_cube_7*.nc')
cubes8 = iris.load('./shitload_of_data/mean_cube_8*.nc')
cubes9 = iris.load('./shitload_of_data/mean_cube_9*.nc')
cubes = iris.cube.CubeList(cubes1+cubes2+cubes3+cubes4+cubes5+cubes6+cubes7+cubes8+cubes9)
print("Not correct:")
print(cubes.concatenate())

print("Correct:")
slices = []
for cube in cubes:
    for yx_slice in cube.slices(['latitude', 'longitude']):
        slices.append(yx_slice)

new_cubes = iris.cube.CubeList(slices)

print(new_cubes.merge())
iris.save(new_cubes[0],'./shitload_of_data/new_cube.nc')
my_plot(new_cubes[0])

In [7]:
#GENERATE DATA
#BE SURE THAT THE DATA IS MERGEABLE:
import numpy as np
hours_in_a_day = 24
n_days = 3

#Cubes will be generated in this range:
t0_glob = 1
tmax_glob = hours_in_a_day * (n_days)
dt_glob = 1 * hours_in_a_day

#Each single cube will have these number of samples and delta t locally:
t_num_samp = 8
dt_loc = dt_glob / t_num_samp

#These are the arrays which will contain t_init and t_max for each cube
t_a = np.arange( t0_glob, tmax_glob - dt_glob + dt_loc,  dt_glob )
t_b = np.arange( t0_glob + dt_glob - dt_loc, tmax_glob, dt_glob )

#We repeat what we did for the time for the rest of the coordinates:

###X
x0_glob = -179
xmax_glob = 180
dx_glob = 35#10

x_num_samp = 10
dx_loc = dx_glob / x_num_samp

x_a = np.arange( x0_glob, xmax_glob - dx_glob + dx_loc,  dx_glob )
x_b = np.arange( x0_glob + dx_glob - dx_loc, xmax_glob, dx_glob )

###Y
y0_glob = -89
ymax_glob = 89
dy_glob = 17#10

y_num_samp = 10
dy_loc = dy_glob / y_num_samp

y_a = np.arange( y0_glob, ymax_glob - dy_glob + dy_loc,  dy_glob )
y_b = np.arange( y0_glob + dy_glob - dy_loc, ymax_glob, dy_glob )


cubes = []
tcubes = len(t_a)
xcubes = len(x_a)
ycubes = len(y_a)
zcubes = 0
for t_init,t_end in zip(t_a,t_b):
    
    for x_init,x_end in zip(x_a,x_b):
    
        for y_init,y_end in zip(y_a,y_b):
    
            cubes.append(generate_cube(tmin = t_init, tmax = t_end,tsamp = t_num_samp,
                                       xmin = x_init, xmax = x_end,xsamp = x_num_samp,
                                       ymin = y_init, ymax = y_end,ysamp = y_num_samp,
                                        zmin = 0, zmax = 7000, zsamp = 2))
            
print(tcubes,xcubes,ycubes)
cubelist = iris.cube.CubeList(cubes)
print(len(cubelist))
print(len(cubelist.concatenate()))
for i in range(len(cubelist)):
    iris.save(cubelist[i],"./shitload_of_data/file_{:03d}.nc".format(i))
print("files saved")

3 10 10
300
1
files saved


In [8]:
#MAP PHASE:
#BE SURE THAT THE MEAN IS DONE PROPERLY:

input_dir = "./shitload_of_data/"
mapper_output_dir = "./mapper_output_dir/"


def get_mean_height(hdfs_path,local_path,save_to):
    cubes = iris.load(local_path)
    mean_height = cubes[0].collapsed(['height','time'], iris.analysis.MEAN)
    iris.save(mean_height,save_to)
    return

#This is done by all nodes in the cluster in parallel:
for root,dirs,files in os.walk(input_dir):
    for f in files:
        hdfs_path = "/".join([root,f])
        local_path = "/".join([root,f])
        save_to = mapper_output_dir+"/"+"map_"+hdfs_path.split("/")[-1].rpartition(".")[-3]+".nc"
        get_mean_height(hdfs_path,local_path,save_to)
#        sys.stdout.write(save_to)

print(iris.load_cube(hdfs_path))
print(iris.load_cube(save_to))


air_temperature / (celsius)         (time: 8; longitude: 10; latitude: 10; height: 2)
     Dimension coordinates:
          time                           x             -             -           -
          longitude                      -             x             -           -
          latitude                       -             -             x           -
          height                         -             -             -           x
     Attributes:
          Conventions: CF-1.5
air_temperature / (celsius)         (longitude: 10; latitude: 10)
     Dimension coordinates:
          longitude                           x             -
          latitude                            -             x
     Scalar coordinates:
          height: 3500.0 metres, bound=(-3500.0, 10500.0) metres
          time: 2000-01-02 11:30:00, bound=(2000-01-01 23:30:00, 2000-01-02 23:30:00)
     Attributes:
          Conventions: CF-1.5
     Cell methods:
          mean: height, time


In [12]:
reducer_output_dir = "./reducer_output_dir"

def concatenate_and_merge(mapper_output_dir,save_to):

    #Load cubes from files inside mapper output dir:
    cubes = []
    for root,dirs,files in os.walk(mapper_output_dir):
        for f in files:
            try:
                to_load = "/".join([root,f])
                cubes.append(iris.load_cube(to_load))
                
            except:
                print("cube not loaded from {}".format(to_load))
                pass
        break

    #if no cubes were loaded, stop.
    if cubes == []:
        print("no files found inside {}".format(mapper_output_dir))
        return

    #Now we have a cube list with only one value for at least one coordinate:
    cube_list = iris.cube.CubeList(cubes)
    #print("Orig:")
    #print(cube_list)
    
    #concatenate
    concatenated_cubes = cube_list.concatenate()
    print("Conc:")
    print(concatenated_cubes)

    #We need to merge them:
    merged_cubes = concatenated_cubes.merge()  
    #print(merged_cubes)
    print("Merged:")
    print(merged_cubes)
    
    #Take one last time the mean:
    print("taking the mean...")
    mean_ = merged_cubes[0].collapsed(['time','height'], iris.analysis.MEAN)

    print("saving...")
    iris.save(mean_,save_to)
    return


save_to = reducer_output_dir+"/reduce_0.nc"
    
concatenate_and_merge(mapper_output_dir,save_to)

sys.stdout.write(save_to)
print("")


Conc:
0: air_temperature / (celsius)         (longitude: 100; latitude: 100)
1: air_temperature / (celsius)         (longitude: 100; latitude: 100)
2: air_temperature / (celsius)         (longitude: 100; latitude: 100)
Merged:
0: air_temperature / (celsius)         (time: 3; longitude: 100; latitude: 100)
taking the mean...
saving...
./reducer_output_dir/reduce_0.nc
